# Example: Principal Components Analysis

We have a collection of points $\{x_1,x_2,..,x_m\} \in \Bbb{R}^{m}, \  \forall i \in \{1,..,m\} \ x_{i} \in \Bbb{R}^{n \times 1}$

And we want to apply lossy compression to these points, meaning we will reduce their dimensionality resulting in less required memory but also less precision. when decoding the compressed points back to their original form.

One way to do this:

The Encoding Function:
$$f: \Bbb{R}^{n \times 1} \to \Bbb{R}^{l \times 1} \\ x \to c$$
With $l<n$.

And for the decoding function, we choose:

$$g: \Bbb{R}^{l \times 1} \to \Bbb{R}^{n \times 1} \\ c \to Dc \approx x$$

with $D \in \Bbb{R}^{n \times l}$.

We Constrain:
* $\forall i \   D_{:,i}$ to be orthogonal.
* $\forall i \   D_{:,i} $ to have unit Norm.

## 1.Finding $ \ f$

The first thing to do is to find the optimal $f$ for all $x$.

One way to do this is to minimize the distance between the input point $x$ and its reconstruction $g(c^{*})$, We can measure this distance using the Norm, in PCA, we use $L^{2}$:

$$c^{*}=arg_{c}min \lVert x - g(c) \rVert_{2} \ / \ c=f(x)$$

So we are looking for $f$ by minimizing the Compressions Loss.

Because $L^2$ is non-negative and $x \to x^2$ is monotonically increasing for positive arguments, we can do this:

$$c^{*}=arg_{c}min \lVert x-g(c) \rVert^{2}_2$$

We minimize:

$$G(c) = (x-g(c))^{T}(x-g(c))\\
G(c) = x^{T}x - x^{T}g(c) - g(c)^{T}x + g(c)^{T}g(c)\\
G(c) = x^{T}x - 2x^{T}g(c) + g(c)^{T}g(c)$$

We remove the first term of $G(c)$ because it doesn't depend on $c$:
$$G(c) = - 2x^{T}g(c) + g(c)^{T}g(c)\\
G(c)=(Dc)^{T}(Dc) - 2x^{T}(Dc)\\
G(c)=c^{T}D^{T}Dc - 2x^{T}Dc\\
G(c)=c^{T}c - 2x^{T}Dc$$

We can minimize $G$ using Vector Calculus:

$$\nabla_{c}(c^{T}c - 2x^{T}Dc)=0\\
2c - 2D^{T}x=0\\
c = D^{T}x$$

So we have:
$$f(x)=D^{T}x$$

Using a further Matrix multiplication, we can also define the PCA reconstruction operation:

$$r(x)=g(f(x))=DD^{T}x$$

## 2.Finding $D$

Next step is to choose the encoding matrix $D$:

Since we'll use the same matrix $D$ to decode all points, we can no longer consider the points in isolation.

We will minimize the Frobenius Norm of the matrix of errors computer over all dimensions and all points (Total Loss):

$$D^{*}=arg_{D}min \sqrt{\sum_{i,j}(x^{(i)}_j -r(x^{(i)})_j)^2}$$

To derive the algorithm for finding $D^*$, we start by considering $l=1$ meaning $D \in \Bbb{R}^{n \times 1}$ is a vector, $d$:

$$d^{*}=arg_dmin \sum_{i} \lVert x^{(i)} - dd^{T}x^{(i)} \rVert^{2}_{2}\\
d^{*}=arg_dmin \sum_{i} \lVert x^{(i)} - (x^{(i)})^Tdd \rVert^{2}_{2}
$$

Let $X \in \Bbb{R}^{m \times n} \ , \ X_{i,:} = (x^{(i)})^T$:

We can Now rewrite the Problem as:

$$d^{*}=arg_dmin \ \lVert X - Xdd^T \rVert^{2}_{F} \\
d^{*}=arg_dmin \ \{-2Tr(X^{T}Xdd^{T}) + Tr(dd^{T}X^{T}Xdd^{T})\} \\
d^{*}=arg_dmax \ Tr(d^{T}X^{T}Xd)
$$

This Optimization problem is may be solved using eigen decomposition. 

Specifically, the Optimal $d$ is given by the eigen vector of $X^{T}X$ Corresponding to the Largest Eigenvalue.

---

## PCA: Sklearn Example

We will try to find $d$ in the simplest scenario, we'll calculate it using numpy, Let's do it:

We have $\{(-1,-1);(-2,-1);(-3,-2);(1,1);(2,1);(3,2)\} \in \Bbb{R}^{2} \ / \ x_1, x_2 \in \Bbb{R}^{2,1}$, $l=1$, So:

$D=d=\begin{bmatrix} d_1 \\ d_2 \end{bmatrix}  \in \Bbb{R}^{2 \times 1}$, 
and $X=\begin{bmatrix} -1 & -1 \\ -2 & -1 \\ -3 & -2 \\ 1 & 1 \\ 2 & 1 \\ 3 & 2 \end{bmatrix}$, and finally:

$$f: \Bbb{R}^{2 \times 1} \to \Bbb{R}^{1 \times 1} \\ x \to d^{T}x$$

And:

$$g: \Bbb{R}^{1 \times 1} \to \Bbb{R}^{2 \times 1} \\ c \to dc$$

We want to find $d$ to minimize the the distance between the reconstructions $dd^{T}x$ and the original points $x$:

## Numpy Way

In [129]:
import numpy as np
import sklearn
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [130]:
X_ = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]]);X_

array([[-1, -1],
       [-2, -1],
       [-3, -2],
       [ 1,  1],
       [ 2,  1],
       [ 3,  2]])

We center the matrix:

In [131]:
X__ = X_ - X_.mean(axis=1)[:, np.newaxis]

Then we calculate $X^{T}X$:

In [132]:
cov = np.matmul(X__.T, X__)

In [133]:
eig_values, eig_vectors = np.linalg.eig(cov)

In [134]:
eig_values

array([2., 0.])

The eigen vector with the largest eigen value is:

In [135]:
eig_vectors[0]

array([0.70710678, 0.70710678])

Now we reconstruct the the vectors:

In [136]:
for x in X:
    print(eig_vectors[0].dot(eig_vectors[0].T.dot(x)))
    print(eig_vectors[0].dot(eig_vectors[0].T.dot(x)))

[-1. -1.]
[-1. -1.]
[-1.5 -1.5]
[-1.5 -1.5]
[-2.5 -2.5]
[-2.5 -2.5]
[1. 1.]
[1. 1.]
[1.5 1.5]
[1.5 1.5]
[2.5 2.5]
[2.5 2.5]


---

## Sklearn
to automate the process of dimensionality reduction, we can simply use Sklearn:

In [137]:
X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]]);X

array([[-1, -1],
       [-2, -1],
       [-3, -2],
       [ 1,  1],
       [ 2,  1],
       [ 3,  2]])

In [138]:
pca = PCA(n_components=1)

In [139]:
# we scale X as we did using Numpy.
pca.fit(StandardScaler().fit_transform(X))

/Users/mohamedakramzaytar/.envs/kaggle/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


PCA(copy=True, iterated_power='auto', n_components=1, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [140]:
for x in X:
    print(pca.inverse_transform(pca.transform([x]))[0])

[-1. -1.]
[-1.5 -1.5]
[-2.5 -2.5]
[1. 1.]
[1.5 1.5]
[2.5 2.5]
